**Корректность проверена на Python 3.7:**
+ pandas 0.23.0
+ numpy 1.14.5
+ scipy 1.1.0

# Множественная проверка гипотез

In [1]:
import pandas as pd
import numpy as np

from scipy.stats import pearsonr
from statsmodels.sandbox.stats.multicomp import multipletests 

/Users/bs/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
import scipy
print(np.__version__)
print(pd.__version__)
print(scipy.__version__)

1.18.2
1.0.3
1.4.1


## Foodmart product sales 

In [3]:
sales = pd.read_csv('foodmart.sales.tsv', sep = '\t', header = 0, parse_dates = [2])

In [4]:
sales.head()

,product_id,store_id,date,sales
0,4,6,1997-01-01,4
1,25,6,1997-01-01,3
2,48,6,1997-01-01,3
3,76,6,1997-01-01,4
4,119,6,1997-01-01,3


In [5]:
products = pd.read_csv('foodmart.products.tsv', sep = '\t', header = 0)

In [6]:
products.head()

,product_class_id,product_id,brand_name,product_name,SKU,SRP,gross_weight,net_weight,recyclable_package,low_fat,units_per_case,cases_per_pallet,shelf_width,shelf_height,shelf_depth
0,30,1,Washington,Washington Berry Juice,90748583674,2.85,8.39,6.39,False,False,30,14,16.9,12.60,7.40
1,52,2,Washington,Washington Mango Drink,96516502499,0.74,7.42,4.42,False,True,18,8,13.4,3.71,22.60
2,52,3,Washington,Washington Strawberry Drink,58427771925,0.83,13.10,11.10,True,True,17,13,14.4,11.00,7.77
3,19,4,Washington,Washington Cream Soda,64412155747,3.64,10.60,9.60,True,False,26,10,22.9,18.90,7.93
4,19,5,Washington,Washington Diet Soda,85561191439,2.19,6.66,4.65,True,False,7,10,20.7,21.90,19.20


In [7]:
sales = sales.merge(products[['product_id', 'product_name']], 
                    on = ['product_id'], how = 'inner')

In [8]:
sales.head()

,product_id,store_id,date,sales,product_name
0,4,6,1997-01-01,4,Washington Cream Soda
1,4,7,1997-01-05,3,Washington Cream Soda
2,4,6,1997-01-06,2,Washington Cream Soda
3,4,17,1997-01-11,2,Washington Cream Soda
4,4,24,1997-01-11,2,Washington Cream Soda


## Корреляция Пирсона

In [9]:
sparse_sales = pd.pivot_table(sales, values='sales', index=['date', 'store_id'],
                     columns=['product_name'], fill_value = 0)

In [10]:
sparse_sales.head()

product_name         ADJ Rosy Sunglasses  Akron City Map  \
date       store_id                                        
1997-01-01 6                           0               0   
           14                          0               0   
1997-01-02 11                          0               0   
           23                          0               0   
1997-01-03 7                           0               0   

product_name         Akron Eyeglass Screwdriver  American Beef Bologna  \
date       store_id                                                      
1997-01-01 6                                  0                      0   
           14                                 0                      0   
1997-01-02 11                                 0                      0   
           23                                 0                      0   
1997-01-03 7                                  0                      0   

product_name         American Chicken Hot Dogs  American Cole Slaw  \
date       store_id                                                  
1997-01-01 6                                 0                   0   
           14                                0                   0   
1997-01-02 11                                0                   0   
           23                                0                   0   
1997-01-03 7                                 0                   0   

product_name         American Corned Beef  American Foot-Long Hot Dogs  \
date       store_id                                                      
1997-01-01 6                            0                            0   
           14                           0                            0   
1997-01-02 11                           0                            0   
           23                           0                            0   
1997-01-03 7                            0                            0   

product_name         American Low Fat Bologna  American Low Fat Cole Slaw  \
date       store_id                                                         
1997-01-01 6                                0                           0   
           14                               0                           0   
1997-01-02 11                               0                           0   
           23                               0                           0   
1997-01-03 7                                0                           0   

product_name         ...  Washington Apple Juice  Washington Berry Juice  \
date       store_id  ...                                                   
1997-01-01 6         ...                       0                       0   
           14        ...                       0                       0   
1997-01-02 11        ...                       0                       0   
           23        ...                       0                       0   
1997-01-03 7         ...                       0                       4   

product_name         Washington Cola  Washington Cranberry Juice  \
date       store_id                                                
1997-01-01 6                       0                           0   
           14                      0                           0   
1997-01-02 11                      0                           0   
           23                      0                           0   
1997-01-03 7                       0                           0   

product_name         Washington Cream Soda  Washington Diet Cola  \
date       store_id                                                
1997-01-01 6                             4                     0   
           14                            0                     0   
1997-01-02 11                            0                     0   
           23                            0                     0   
1997-01-03 7                             0                     0   

product_name         Wa

In [11]:
%%time 
corr_data = []

for i, lhs_column in enumerate(sparse_sales.columns):
    for j, rhs_column in enumerate(sparse_sales.columns):
        if i >= j:
            continue
        
        corr, p = pearsonr(sparse_sales[lhs_column], sparse_sales[rhs_column])
        corr_data.append([lhs_column, rhs_column, corr, p])

CPU times: user 2min 16s, sys: 1.19 s, total: 2min 17s
Wall time: 2min 21s


In [12]:
sales_correlation = pd.DataFrame.from_records(corr_data)
sales_correlation.columns = ['product_A', 'product_B', 'corr', 'p']

In [13]:
sales_correlation.head()

,product_A,product_B,corr,p
0,ADJ Rosy Sunglasses,Akron City Map,0.076608,0.032414
1,ADJ Rosy Sunglasses,Akron Eyeglass Screwdriver,-0.006581,0.854396
2,ADJ Rosy Sunglasses,American Beef Bologna,0.038685,0.280546
3,ADJ Rosy Sunglasses,American Chicken Hot Dogs,0.041105,0.251529
4,ADJ Rosy Sunglasses,American Cole Slaw,-0.045887,0.200484


Сколько гипотез об отсутствии корреляции отвергается без поправки на множественную проверку?

In [14]:
(sales_correlation.p < 0.05).value_counts()

False    982453
True     232008
Name: p, dtype: int64

## Поправка на множественную проверку

### Метод Холма

In [15]:
reject, p_corrected, a1, a2 = multipletests(sales_correlation.p, 
                                            alpha = 0.05, 
                                            method = 'holm') 

In [16]:
sales_correlation['p_corrected'] = p_corrected
sales_correlation['reject'] = reject

In [17]:
sales_correlation.head()

,product_A,product_B,corr,p,p_corrected,reject
0,ADJ Rosy Sunglasses,Akron City Map,0.076608,0.032414,1.0,False
1,ADJ Rosy Sunglasses,Akron Eyeglass Screwdriver,-0.006581,0.854396,1.0,False
2,ADJ Rosy Sunglasses,American Beef Bologna,0.038685,0.280546,1.0,False
3,ADJ Rosy Sunglasses,American Chicken Hot Dogs,0.041105,0.251529,1.0,False
4,ADJ Rosy Sunglasses,American Cole Slaw,-0.045887,0.200484,1.0,False


In [18]:
sales_correlation.reject.value_counts()

False    1212733
True        1728
Name: reject, dtype: int64

In [19]:
sales_correlation[sales_correlation.reject == True].sort_values(by='corr', ascending=False)

,product_A,product_B,corr,p,p_corrected,reject
1063670,Just Right Vegetable Soup,Plato French Roast Coffee,0.340598,1.226033e-22,1.488970e-16,True
885574,Great Muffins,Nationeel Grape Fruit Roll,0.322176,2.688803e-20,3.265443e-14,True
473067,Club Low Fat Cottage Cheese,Skinner Strawberry Drink,0.306701,1.883995e-18,2.288034e-12,True
1181001,Robust Monthly Home Magazine,Tri-State Lemons,0.303269,4.674973e-18,5.677558e-12,True
1160248,Pleasant Regular Ramen Soup,Shady Lake Ravioli,0.298502,1.619119e-17,1.966350e-11,True
...,...,...,...,...,...,...
662127,Ebony Potatos,Just Right Fancy Canned Anchovies,0.194914,4.073740e-08,4.940380e-02,True
565740,Cutting Edge Cole Slaw,Horatio Potato Chips,0.194875,4.099170e-08,4.971215e-02,True
615865,Discover Ravioli,High Quality Soft Napkins,0.194869,4.103075e-08,4.975947e-02,True
559150,Cormorant Toilet Bowl Cleaner,Plato Chunky Peanut Butter,0.194863,4.107241e-08,4.980995e-02,True


### Метод Бенджамини-Хохберга

In [20]:
reject, p_corrected, a1, a2 = multipletests(sales_correlation.p, 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 

In [21]:
sales_correlation['p_corrected'] = p_corrected
sales_correlation['reject'] = reject

In [22]:
sales_correlation.head()

,product_A,product_B,corr,p,p_corrected,reject
0,ADJ Rosy Sunglasses,Akron City Map,0.076608,0.032414,0.203716,False
1,ADJ Rosy Sunglasses,Akron Eyeglass Screwdriver,-0.006581,0.854396,0.956078,False
2,ADJ Rosy Sunglasses,American Beef Bologna,0.038685,0.280546,0.630699,False
3,ADJ Rosy Sunglasses,American Chicken Hot Dogs,0.041105,0.251529,0.600790,False
4,ADJ Rosy Sunglasses,American Cole Slaw,-0.045887,0.200484,0.541916,False


In [23]:
sales_correlation.reject.value_counts()

False    1138407
True       76054
Name: reject, dtype: int64

In [24]:
sales_correlation[sales_correlation.reject == True].sort_values(by='corr', ascending=False)

,product_A,product_B,corr,p,p_corrected,reject
1063670,Just Right Vegetable Soup,Plato French Roast Coffee,0.340598,1.226033e-22,1.488970e-16,True
885574,Great Muffins,Nationeel Grape Fruit Roll,0.322176,2.688803e-20,1.632723e-14,True
473067,Club Low Fat Cottage Cheese,Skinner Strawberry Drink,0.306701,1.883995e-18,7.626793e-13,True
1181001,Robust Monthly Home Magazine,Tri-State Lemons,0.303269,4.674973e-18,1.419393e-12,True
1160248,Pleasant Regular Ramen Soup,Shady Lake Ravioli,0.298502,1.619119e-17,3.932713e-12,True
...,...,...,...,...,...,...
108018,Best Choice Dried Dates,Tell Tale Party Nuts,0.105665,3.130716e-03,4.999516e-02,True
419799,Carrington Frozen Carrots,Sphinx Muffins,0.105664,3.130854e-03,4.999649e-02,True
1184578,Skinner Diet Cola,Special Wheat Puffs,0.105664,3.130881e-03,4.999649e-02,True
821999,Gauss Monthly Fashion Magazine,PigTail Beef TV Dinner,0.105664,3.130969e-03,4.999724e-02,True
